In [33]:
from dotenv import load_dotenv
load_dotenv()

from flask import Flask, render_template, request, jsonify
from werkzeug.utils import secure_filename
import os

from inject_relationship import execute_ultra_optimized_relationships
from inject_node import execute_ultra_optimized_nodes
from schema import schema_modelling
from sqlite_to_csv import export_sqlite_to_csv
from erd_text import erd_text_generate
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from neo4j import GraphDatabase
from langraph_neo4j3 import AgentState, run_agent_workflow
from langchain_neo4j import Neo4jGraph
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Initialize LLM
os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY4"]
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=1,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
)

driver = None
db_name = "sales_master"
modelling_output = None
neo4j_graph = None

In [4]:
uri="neo4j+s://2d5e8539.databases.neo4j.io"
username="neo4j"
password="xn8iCGEj2vymA3-43-57qlL63CD70SthzTE_Mt8QfG0"
driver = GraphDatabase.driver(uri, auth=(username, password))
neo4j_graph = Neo4jGraph(
    url=uri,
    username=username,
    password=password,
    database="neo4j", 
    driver_config={
        "max_connection_lifetime": 3000,  # 50 minutes
        "keep_alive": True,
        "max_connection_pool_size": 50
    },
    enhanced_schema=True)

In [35]:
from graph import make_graph_html

replacement = export_sqlite_to_csv(db_name+".sqlite",db_name+"_files")
print(replacement)
_, erd_text = erd_text_generate(db_name, replacement)
print(_)
print(erd_text)
modelling_output = schema_modelling(erd_text, llm)
# graph_html_path = make_graph_html(modelling_output)

Processing table: customer_table
Saved customer_table -> sales_master_files\customer_table.csv
Processing table: order_table
Saved order_table -> sales_master_files\order_table.csv
Processing table: product_table
Saved product_table -> sales_master_files\product_table.csv
Processing table: order_product_table
Saved order_product_table -> sales_master_files\order_product_table.csv
All tables exported!
{}
{'customer_table': {'columns': [{'name': 'CUSTOMERNAME', 'type': 'TEXT'}, {'name': 'PHONE', 'type': 'TEXT'}, {'name': 'ADDRESSLINE1', 'type': 'TEXT'}, {'name': 'ADDRESSLINE2', 'type': 'TEXT'}, {'name': 'CITY', 'type': 'TEXT'}, {'name': 'STATE', 'type': 'TEXT'}, {'name': 'POSTALCODE', 'type': 'TEXT'}, {'name': 'COUNTRY', 'type': 'TEXT'}, {'name': 'TERRITORY', 'type': 'TEXT'}, {'name': 'CONTACTLASTNAME', 'type': 'TEXT'}, {'name': 'CONTACTFIRSTNAME', 'type': 'TEXT'}], 'primary_keys': ['CUSTOMERNAME'], 'foreign_keys': []}, 'order_table': {'columns': [{'name': 'ORDERNUMBER', 'type': 'INTEGER

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]